<a target="_blank" href="https://colab.research.google.com/github/ChuBL/How-to-Use-Mindat-API/blob/main/How_to_Use_Mindat_API.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# How to Use OpenMindat Data API to Query and Download Datasets


## 0. Access Your Mindat API Token

[How to Get My Mindat API Key or Token?](https://www.mindat.org/a/how_to_get_my_mindat_api_key)

## 1. Dependencies (Please run this section first)

In [ ]:
from pathlib import Path
import os
import sys
import json
import re
import pprint
import requests

In [ ]:
# You can change this working path according to your preference
# If the folder didn't show up in the left column, click the refresh button
WORKING_DIR = "/content/MindatAPI_folder/"
Path(WORKING_DIR).mkdir(parents=True, exist_ok=True)

You should **avoid** placing your API token explicitly in your code if you plan to share it.

For example, you can drag a text file of your token to the working directory to upload it, then access it using the following code. Or you can input it manually.

You should find your API key at [mindat.org](https://www.mindat.org/) >> `My Home Page` >> `Edit My Page`, at the bottom of the page.


In [ ]:
YOUR_API_KEY = ""

In [1]:
%%script false --no-raise-error
# comment out the first line to activate this code block
YOUR_API_KEY = input()

In [ ]:
try:
    assert 0 != len(YOUR_API_KEY)
except AssertionError:
    raise Exception("Please set a valid API token before the start!")

## 2. Use Cases


### Get the Geomaterial Items with Selected Fields


Examples for Mindat API fields: `fields=id,longid,guid,name,updttime,mindat_formula,mindat_formula_note,ima_formula,ima_status,...`

For entire fields list, see: [OpenMindat API Documentation](https://api.mindat.org/schema/redoc/#tag/items/operation/items_list)

A comprehensive field explanations document can be found [here](https://github.com/smrgeoinfo/How-to-Use-Mindat-API/blob/main/geomaterialfields.csv).

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

select_file_name = "mindat_items_select.json"
select_file_path = Path(WORKING_DIR, select_file_name)
select_file_path

PosixPath('/content/MindatAPI_folder/mindat_items_select.json')

In [ ]:
# set your selected fields here
fields_str = 'id,name,mindat_formula'

with open(select_file_path, 'w') as f:
    params = {
        'fields': fields_str, # put your selected fields here
        'format': 'json'
    }
    response = requests.get(MINDAT_API_URL+"/geomaterials/",
                    params=params,
                    headers=headers)

    result_data = response.json()["results"]
    json_data = {"results": result_data}

    # Uncomment the following code to obtain more than ten results
    # while True:
    #     try:
    #         next_url = response.json()["next"]
    #         response = requests.get(next_url, headers=headers)
    #         json_data["results"] += response.json()['results']

    #     except requests.exceptions.MissingSchema as e:
    #         # This error indicates the `next_url` is none
    #         break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data['results'])) + " entries to " + str(select_file_path))

### Get the Geomaterial Items with Omitted Fields

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

omit_file_name = "mindat_items_omit.json"
omit_file_path = Path(WORKING_DIR, omit_file_name)
omit_file_path

PosixPath('/content/MindatAPI_folder/mindat_items_omit.json')

In [ ]:
omit_str = 'id,name,updttime'


with open(omit_file_path, 'w') as f:
    params = {
        'omit': omit_str,
        'format': 'json'
    }
    response = requests.get(MINDAT_API_URL+"/geomaterials/",
                    params=params,
                    headers=headers)

    result_data = response.json()["results"]
    json_data = {"results": result_data}

    # Uncomment the following code to obtain more than ten results
    # while True:
    #     try:
    #         next_url = response.json()["next"]
    #         response = requests.get(next_url, headers=headers)
    #         json_data["results"] += response.json()['results']

    #     except requests.exceptions.MissingSchema as e:
    #         # This error indicates the `next_url` is none
    #         break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data['results'])) + " entries to " + str(omit_file_path))

Successfully saved 10 entries to /content/MindatAPI_folder/mindat_items_omit.json


### Get the IMA-Approved Mineral Geomaterial Items

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

ima_file_name = "mindat_items_IMA.json"
ima_file_path = Path(WORKING_DIR, ima_file_name)
ima_file_path

PosixPath('/content/MindatAPI_folder/mindat_items_IMA.json')

This query will obtain thousands of mineral species and may take a couple of minutes(4min ~).

In [ ]:
with open(ima_file_path, 'w') as f:
    params = {
        'format': 'json'
    }
    response = requests.get(MINDAT_API_URL+"/minerals_ima/",
                    params=params,
                    headers=headers)

    result_data = response.json()["results"]
    json_data = {"results": result_data}

    while True:
        try:
            next_url = response.json()["next"]
            response = requests.get(next_url, headers=headers)
            json_data["results"] += response.json()['results']

        except requests.exceptions.MissingSchema as e:
            # This error indicates the `next_url` is none
            break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data['results'])) + " entries to " + str(ima_file_path))

Successfully saved 5943 entries to /content/MindatAPI_folder/mindat_items_IMA.json


### Get the Geomaterial Items with Filters

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

filter_file_name = "mindat_items_filter.json"
filter_file_path = Path(WORKING_DIR, filter_file_name)
filter_file_path

PosixPath('/content/MindatAPI_folder/mindat_items_filter.json')

This query will obtain full-sized IMA mineral information from Mindat and may take a couple of minutes(6min ~).

In [ ]:
filter_dict = {
          "ima_status": [
                    "APPROVED"
                ], # an alternative way to obtain the IMA-approved minerals
          #"id_in": '1,2',
          #"colour": "green",
          'format': 'json',
          }

with open(filter_file_path, 'w') as f:

    response = requests.get(MINDAT_API_URL+"/geomaterials/",
                    params=filter_dict,
                    headers=headers)

    result_data = response.json()["results"]
    json_data = {"results": result_data}

    while True:
        try:
            next_url = response.json()["next"]
            response = requests.get(next_url, headers=headers)
            json_data["results"] += response.json()['results']

        except requests.exceptions.MissingSchema as e:
            # This error indicates the `next_url` is none
            break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data["results"])) + " entries to " + str(filter_file_path))

Successfully saved 5943 entries to /content/MindatAPI_folder/mindat_items_filter.json


### Get the Geomaterial Items with Fuzzy Search

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

file_name = "mindat_items_search.json"
file_path = Path(WORKING_DIR, file_name)
file_path

PosixPath('/content/drive/MyDrive/MindatAPI_folder/mindat_items_search.json')

In [ ]:
filter_dict = {'q': 'quartz and ruby', # input what you about to search same as the home page
          'format': 'json',
          }

with open(file_path, 'w') as f:
    params = filter_dict

    response = requests.get(MINDAT_API_URL+"/geomaterials_search/",
                    params=params,
                    headers=headers)

    result_data = response.json()
    json_data = {"results": result_data}

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data["results"])) + " entries to " + str(file_path))

Successfully saved 197 entries to /content/drive/MyDrive/MindatAPI_folder/mindat_items_search.json


### Get Single Locality

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

sig_loc_file_name = "mindat_loc_single.json"
sig_loc_file_path = Path(WORKING_DIR, sig_loc_file_name)
sig_loc_file_path

PosixPath('/content/drive/MyDrive/MindatAPI_folder/mindat_loc_single.json')

In [ ]:
with open(sig_loc_file_path, 'w') as f:
    # The Mindat locality page for Moscow, Idaho
    # https://www.mindat.org/loc-58150.html
    locality_id = '306797'
    response = requests.get(MINDAT_API_URL + '/localities/' + locality_id,
                    headers=headers)
    result_data = response.json()
    json_data = {"results": [result_data]}

    json.dump(json_data, f, indent=4)
print("Successfully saved the locality to " + str(sig_loc_file_path))

Successfully saved the locality to /content/drive/MyDrive/MindatAPI_folder/mindat_loc_single.json


### Get Locality List with multiple IDs

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

loc_file_name = "mindat_loc_list.json"
loc_file_path = Path(WORKING_DIR, loc_file_name)
loc_file_path

PosixPath('/content/drive/MyDrive/MindatAPI_folder/mindat_loc_list.json')

In [ ]:
with open(loc_file_path, 'w') as f:
    params = {
        # Mindat loc ids for Moscow and Los Angeles
        'id__in': '58150,3493',
        'format': 'json'
    }

    response = requests.get(MINDAT_API_URL + '/localities/',
                    params=params,
                    headers=headers)

    result_data = response.json()["results"]
    json_data = {"results": result_data}

    while True:
        try:
            next_url = response.json()["next"]
            response = requests.get(next_url, headers=headers)
            json_data["results"] += response.json()['results']

        except requests.exceptions.MissingSchema as e:
            # This error indicates the `next_url` is none
            break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data['results'])) + " entries to " + str(loc_file_path))

Successfully saved 2 entries to /content/drive/MyDrive/MindatAPI_folder/mindat_loc_list.json


# 3. Test Zone

### Get All the Geomaterial Items


❗Please note that this section of codes will retrieve hundreds of **MB** of data from the Mindat server.

❗This operation may consume a significant time and API access quota.

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

all_file_name = "mindat_items_all.json"
all_file_path = Path(WORKING_DIR, all_file_name)
all_file_path

In [ ]:
with open(all_file_path, 'w') as f:
    params = {
        'format': 'json',
    }
    response = requests.get(MINDAT_API_URL+"/geomaterials/",
                            params=params,
                            headers=headers)
    result_data = response.json()["results"]
    json_data = {"results": result_data}

    while True:
        try:
            next_url = response.json()["next"]
            response = requests.get(next_url, headers=headers)
            json_data["results"] += response.json()['results']

        except requests.exceptions.MissingSchema as e:
            # This error indicates the `next_url` is none
            break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data['results'])) + " entries to " + str(all_file_path))

### Get the Geomaterial Items with Filters

In [ ]:
MINDAT_API_URL = "https://api.mindat.org"
headers = {'Authorization': 'Token '+ YOUR_API_KEY}

filter_file_name = "mindat_items_filter.json"
filter_file_path = Path(WORKING_DIR, filter_file_name)
filter_file_path

PosixPath('/content/drive/MyDrive/MindatAPI_folder/mindat_items_filter.json')

In [ ]:
filter_dict = {
          #"longid": "1:1:13:6",
          #"colour": "green",
          "guid": "c339183f-cf96-4ccc-96c9-6eeb7a0c81d0",
          'format': 'json',
          }

with open(filter_file_path, 'w') as f:

    response = requests.get(MINDAT_API_URL+"/geomaterials/",
                    params=filter_dict,
                    headers=headers)

    result_data = response.json()['results']
    json_data = {"results": result_data}

    # Uncomment the following code to obtain more than ten results
    # while True:
    #     try:
    #         next_url = response.json()["next"]
    #         response = requests.get(next_url, headers=headers)
    #         json_data["results"] += response.json()['results']

    #     except requests.exceptions.MissingSchema as e:
    #         # This error indicates the `next_url` is none
    #         break

    json.dump(json_data, f, indent=4)
print("Successfully saved " + str(len(json_data["results"])) + " entries to " + str(filter_file_path))

Successfully saved 10 entries to /content/drive/MyDrive/MindatAPI_folder/mindat_items_filter.json
